# AI-LAB SESSION 5: Model-Free Reinforcement Learning

In this tutorial we will see some additional functionalities available to OpenAI Gym environments

## Cliff environment

The environment used is **Cliff** (taken from the book of Sutton and Barto as visible in the figure)
![CliffWalking](images/cliff.png)

The agent starts in cell $(3, 0)$ and has to reach the goal in $(3, 11)$. Falling from the cliff resets the position to the start state (the episode ends only when the goal state is reached). All other cells are safe. Action dinamycs is deterministic, meaning that the agent always reaches the desired next state.

## Assignment 1

Your first assignment is to implement the *Q-Learning* algorithm on **Cliff**. In particular you need to implement both $\epsilon$-greedy and *Softmax* versions for the exploration heuristic. The solution returned must be a tuple *(policy, rewards, lengths)* where:
* *policy*: array of action identifiers where the $i$-th action refers to the $i$-th state
* *rewards*: array of rewards where the $i$-th reward refers to the $i$-th episode of the training performed
* *lengths*: array of lengths where the $i$-th length refers to the $i$-th episode of the training performed (length in number of steps)

Functions to implement:
* *epsilon_greedy(q, state, epsilon)*
* *softmax(softmax(q, state, temp)*
* *q_learning(environment, episodes, alpha, gamma, expl_func, expl_param)*

In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
import numpy as np
from utils.funcs import run_episode, plot, rolling
from timeit import default_timer as timer

It is useful to draw a number given a specific probability distribution. In particular, among the 5 choices, the 3rd is the one that is most likely to be chosen (highest probability value)

In [20]:
np.random.choice(5, p=[0.1, 0.2, 0.5, 0.1, 0.1])

2

The following functions have to be implemented

In [21]:
def epsilon_greedy(q, state, epsilon):
    """
    Epsilon-greedy action selection function
    
    Args:
        q: q table
        state: agent's current state
        epsilon: epsilon parameter
    
    Returns:
        action id
    """
    bestAction = np.argmax(q[state])
    if np.random.uniform() <= epsilon:
        bestAction = np.random.choice(len(q[state]))
    return bestAction

In [22]:
def softmax(q, state, temp):
    """
    Softmax action selection function
    
    Args:
    q: q table
    state: agent's current state
    temp: temperature parameter
    
    Returns:
        action id
    """
    p = np.exp(q[state]/temp)
    pSum = np.sum(p)
    p = np.divide(p,pSum)
    return np.random.choice(len(q[state]),p=p)

In [23]:
def q_learning(env, episodes, alpha, gamma, expl_func, expl_param):
    """
    Performs the Q-Learning algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        episodes: number of episodes for training
        alpha: alpha parameter
        gamma: gamma parameter
        expl_func: exploration function (epsilon_greedy, softmax)
        expl_param: exploration parameter (epsilon, T)
    
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """
    rewards = np.zeros(episodes)  # Rewards array
    lengths = np.zeros(episodes)  # Lengths array
    policy = np.random.choice(env.action_space.n, env.observation_space.n)  # Random policy
    #Genero Q
    Q = np.zeros((env.observation_space.n,env.action_space.n))
    for e in range(episodes):
        s = env.reset()
        while True:
            a = expl_func(Q,s,expl_param)
            nextState,r,d, _ = env.step(a) #execute step
            Q[s,a] += alpha*(r + gamma * np.max(Q[nextState]) - Q[s,a])
            s = nextState
            rewards[e] += 1
            lengths[e] = r
            if d: #d true
                break
    for s in range(env.observation_space.n):
        policy[s] = np.argmax(Q[s])
    return policy, rewards, lengths

The following code executes your implementation of *Q-Learning*

In [31]:
envname = "Cliff-v0"

print("\n----------------------------------------------------------------")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

env = gym.make(envname)
env.render()
print()

# Learning parameters
episodes = 500
alpha = .3
gamma = .9
epsilon = .1

t = timer()

# Q-Learning epsilon greedy
policy, rewards, lengths = q_learning(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Execution time: {0}s\nPolicy:\n{1}\n".format(round(timer() - t, 4), np.vectorize(env.actions.get)(policy.reshape(
    env.shape))))
_ = run_episode(env, policy, 20)


----------------------------------------------------------------
	Environment:  Cliff-v0
----------------------------------------------------------------

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T


Execution time: 0.2635s
Policy:
[['U' 'R' 'U' 'R' 'R' 'D' 'R' 'R' 'R' 'R' 'D' 'D']
 ['D' 'R' 'D' 'D' 'R' 'R' 'R' 'R' 'R' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'D']
 ['U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U']]



Correct results for *Q-Learning* can be found [here](results/q_learning_results.txt). Notice that the result is stochastic so the final policy could differ a bit.

## Assignment 2

Your first assignment is to implement the *SARSA* algorithm on **Cliff**. In particular you need to implement both $\epsilon$-greedy and *Softmax* versions for the exploration heuristic (you can reuse the same functions of Assignment 1). The solution returned must be a tuple *(policy, rewards, lengths)* where:
* *policy*: array of action identifiers where the $i$-th action refers to the $i$-th state
* *rewards*: array of rewards where the $i$-th reward refers to the $i$-th episode of the training performed
* *lengths*: array of lengths where the $i$-th length refers to the $i$-th episode of the training performed (length in number of steps)

Functions to implement:
* *SARSA(environment, episodes, alpha, gamma, expl_func, expl_param)*

In [11]:
def sarsa(environment, episodes, alpha, gamma, expl_func, expl_param):
    """
    Performs the SARSA algorithm for a specific environment
    
    Args:
        environment: OpenAI gym environment
        episodes: number of episodes for training
        alpha: alpha parameter
        gamma: gamma parameter
        expl_func: exploration function (epsilon_greedy, softmax)
        expl_param: exploration parameter (epsilon, T)
    
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """
    rewards = np.zeros(episodes)  # Rewards array
    lengths = np.zeros(episodes)  # Lengths array
    policy = np.random.choice(environment.action_space.n, environment.observation_space.n)  # Random policy
    #Genero Q
    Q = np.zeros((environment.observation_space.n,environment.action_space.n))
    for e in range(episodes):
        s = environment.reset()
        a = expl_func(Q,s,expl_param)
        while True:
            nextState,r,d, _ = environment.step(a) #execute step
            aFirst = expl_func(Q,nextState,expl_param)
            Q[s,a] += alpha*(r + gamma * Q[nextState, aFirst] - Q[s,a])
            s = nextState
            aFirst = a
            rewards[e] += 1
            lengths[e] = r
            if d: #d true
                break
    for s in range(environment.observation_space.n):
        policy[s] = np.argmax(Q[s])
    return policy, rewards, lengths

The following code executes your implementation of *SARSA*

In [12]:
envname = "Cliff-v0"

print("\n----------------------------------------------------------------")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

env = gym.make(envname)
env.render()
print()

# Learning parameters
episodes = 500
alpha = .3
gamma = .9
epsilon = .1

t = timer()

# SARSA epsilon greedy
policy, rews, lengths = sarsa(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Execution time: {0}s\nPolicy:\n{1}\n".format(round(timer() - t, 4), np.vectorize(env.actions.get)(policy.reshape(
    env.shape))))
_ = run_episode(env, policy, 20)


----------------------------------------------------------------
	Environment:  Cliff-v0
----------------------------------------------------------------

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T




KeyboardInterrupt: 

Correct results for *SARSA* can be found [here](results/sarsa_results.txt). Notice that the result is stochastic so the final policy could differ a bit.

## Discussion

Now that you have veryfied the results, try to employ *Softmax* instead of $\epsilon$-greedy as exploration heuristic. Are there any significant changes? Why?

## Comparison
The following code performs a comparison between the 3 reinforcement learning algorithms of session 4 and 5: *Model-Based*, *Q-Learning* and *SARSA*. First of all copy your *value_iteration* and *model_based* functions from previous sessions here below

In [6]:
def value_iteration(environment, maxiters, gamma, delta):
    """
    Performs the value iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: max iterations allowed
        gamma: gamma value
        delta: delta value
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    v = np.zeros(environment.observation_space.n)  # Value function init
    iters = 0
    while True:
        iters += 1
        pv = v.copy()  # Previous value function
        v = (environment.T * (environment.R + gamma * v)).sum(axis=2).max(axis=1)  # New value function: max(a) of Q(s, a)
        # Check for convergence
        if iters == maxiters or np.max(np.abs(v - pv)) < delta:
            break
    # Extract policy by argmax(a) of Q(s, a): best action for each state
    return (environment.T * (environment.R + gamma * v)).sum(axis=2).argmax(axis=1)

In [7]:
def model_based(environment, episodes, ep_limit, vmaxiters, gamma, delta):
    """
    Performs the model-based algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        episodes: number of episodes for training
        ep_limit: limit to episode length
        vmaxiters: max iterations allowed for VI
        gamma: gamma value
        delta: delta value
        
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """
    rews = np.zeros(episodes)
    lengths = np.zeros(episodes)
    policy = np.random.choice(environment.action_space.n, environment.observation_space.n)  # Random initial policy
    #policy = np.zeros(environment.observation_space.n, dtype="int16")
    # Counters
    ct = np.zeros((environment.observation_space.n, environment.action_space.n, environment.observation_space.n))
    cr = np.zeros((environment.observation_space.n, environment.action_space.n, environment.observation_space.n))
    for i in range(episodes):
        s = environment.reset()
        iters = 0
        while True:
            sp, r, d, _ = environment.step(policy[s])  # Execute a step
            # Update counters
            ct[s, policy[s], sp] += 1
            cr[s, policy[s], sp] += r
            iters += 1
            rews[i] += r
            if d or iters == ep_limit:
                break
            s = sp
        lengths[i] = iters
        # Update transition model and rewards
        environment.R = cr.copy()
        np.divide(cr, ct, out=environment.R, where=ct != 0)
        ts = ct.sum(axis=2, keepdims=True)
        environment.T = ct.copy()
        np.divide(ct, ts, out=environment.T, where=ts != 0)
        policy = value_iteration(environment, vmaxiters, gamma, delta)  # Compute policy with VI
    return policy, rews, lengths

Now execute the following code and analyze the charts

In [ ]:
envname = "Cliff-v0"

print("\n----------------------------------------------------------------")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

env = gym.make(envname)
env.render()

# Learning parameters
episodes = 500
ep_limit = 50
vmaxiters = 50
alpha = .3
gamma = .9
epsilon = .1
delta = 1e-3

rewser = []
lenser = []

litres = np.arange(1, episodes + 1)  # Learning iteration values
window = 10  # Rolling window
mrew = np.zeros(episodes)
mlen = np.zeros(episodes)

t = timer()

# Model-based
_, rews, lengths = model_based(env, episodes, ep_limit, vmaxiters, gamma, delta)
rews = rolling(rews, window)
rewser.append({"x": np.arange(1, len(rews) + 1), "y": rews, "label": "Model-Based"})
lengths = rolling(lengths, window)
lenser.append({"x": np.arange(1, len(lengths) + 1), "y": lengths, "label": "Model-Based"})

# Q-Learning
_, rews, lengths = q_learning(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
rews = rolling(rews, window)
rewser.append({"x": np.arange(1, len(rews) + 1), "y": rews, "ls": "-", "label": "Q-Learning"})
lengths = rolling(lengths, window)
lenser.append({"x": np.arange(1, len(lengths) + 1), "y": lengths, "ls": "-", "label": "Q-Learning"})

# SARSA
_, rews, lengths = sarsa(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
rews = rolling(rews, window)
rewser.append({"x": np.arange(1, len(rews) + 1), "y": rews, "label": "SARSA"})
lengths = rolling(lengths, window)
lenser.append({"x": np.arange(1, len(lengths) + 1), "y": lengths, "label": "SARSA"})

print("Execution time: {0}s".format(round(timer() - t, 4)))

plot(rewser, "Rewards", "Episodes", "Rewards")
plot(lenser, "Lengths", "Episodes", "Lengths")


----------------------------------------------------------------
	Environment:  Cliff-v0
----------------------------------------------------------------

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



## Discussion

Can you explain the different trends in the charts?

Correct trends can be seen in the following images: yours should be similar

### Rewards
![rewards](results/rewards.png)

### Lengths
![lengths](results/lengths.png)